In [1]:
import pandas as pd
import numpy as np
import math
from pyomo.environ import *
from pyomo.opt import SolverFactory
import os
solver_location = "C:\\Users\\DeepakYadav\\OneDrive - Tata Insights and Quants\\Optimization\\Cutting Stock Problem 1\\cbc"
os.chdir(r'C:\\Users\\DeepakYadav\\OneDrive - Tata Insights and Quants\\Optimization\\Cutting Stock Problem 1')

In [2]:
#DataReading
Mcdf = pd.read_excel('Cutting Stock Data 3.xlsx',sheet_name = 'Mother Coil')
Scdf = pd.read_excel('Cutting Stock Data 3.xlsx',sheet_name = 'Slitted Coil Demand')

In [3]:
#Columnsrenaming
Mcdf.rename(columns = {'Mother Coil Width': 'McWidth','Cost Per Coil (Rs)' : 'McCost', 'Mother Coil Weight (Tons)': 'McWeight'},inplace = True)
Scdf.rename(columns = {'Slitted Coil Width': 'ScWidth','Demand (tons)' : 'Demand'},inplace = True)

In [4]:
#DataProcessing
Scdf['Index'] = Scdf['ScWidth']
Mcdf['Index'] = Mcdf['McWidth']
Scdf['ScDemand'] = Scdf['ScWidth'] / Mcdf['McWidth'][0]
Scdf['ScDemand'] = Scdf['ScDemand'] * Mcdf['McWeight'][0]
Scdf['ScDemand'] = round(Scdf['Demand']/ Scdf['ScDemand'])
Scdf['ScDemand'] = Scdf['ScDemand'].astype(int)

In [5]:
#DefiningIndexes
Scdf.set_index(['Index'],inplace = True)
Mcdf.set_index(['Index'],inplace = True)

In [6]:
#DefiningSets
s_ScType = list(Scdf.index)
s_McSerialNo = ['sno - '+str(x) for x in range(1,21)]
s_McType = list(Mcdf.index)

In [7]:
#Defining Parameters
p_ScDemand = Scdf['ScDemand'].to_dict()
p_ScWidth = Scdf['ScWidth'].to_dict()
p_McWidth = Mcdf['McWidth'].to_dict()
p_McCost = Mcdf['McCost'].to_dict()

In [8]:
#DefiningModel
model = ConcreteModel()

#Defining Variables
model.v_mothcoil = Var(s_McType,s_McSerialNo,s_ScType, domain = NonNegativeIntegers)
model.v_mothcoilusd = Var(s_McType,s_McSerialNo, domain = Binary)

In [9]:
#Defining Constraints
def demand_rule(model,s):
    return sum(model.v_mothcoil[k,m,s] for m in s_McSerialNo for k in s_McType) == p_ScDemand[s]
model.demand = Constraint(s_ScType, rule = demand_rule)
    
def Capacity_rule(model,j,k):
    return sum(model.v_mothcoil[j,k,i] * p_ScWidth[i] for i in s_ScType) <= p_McWidth[j] * model.v_mothcoilusd[j,k]
model.capacity_constraint = Constraint(s_McType,s_McSerialNo, rule = Capacity_rule)

In [10]:
#Defining Objective
model.cost = Objective(
expr=sum(model.v_mothcoilusd[i,j] * p_McCost[i] for i in s_McType for j in s_McSerialNo),
sense=minimize
)

In [11]:
#Run Model
optimizer = SolverFactory("cbc", executable = solver_location)
# optimizer.options['ratio'] = 0.05
results = optimizer.solve(model,tee=True) 
results.write()
print("Optimal Solution:")

Welcome to the CBC MILP Solver 
Version: 2.10 
Build Date: Oct 13 2019 

command line - C:\Users\DeepakYadav\OneDrive - Tata Insights and Quants\Optimization\Cutting Stock Problem 1\cbc.exe -printingOptions all -import C:\Users\DEEPAK~1\AppData\Local\Temp\tmpcrfwv8ft.pyomo.lp -stat=1 -solve -solu C:\Users\DEEPAK~1\AppData\Local\Temp\tmpcrfwv8ft.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
Presolve 45 (0) rows, 240 (0) columns and 440 (0) elements
Statistics for presolved model
Original problem has 240 integers (40 of which binary)
==== 200 zero objective 3 different
200 variables have objective of 0
20 variables have objective of 7425
20 variables have objective of 7750
==== absolute objective values 3 different
200 variables have objective of 0
20 variables have objective of 7425
20 variables have objective of 7750
==== for integers 200 zero objective 3 different
200 variables have objective of 0
20 variables have objective of 7425
20 variables

In [12]:
#Post Process
for i in s_McType:
        for j in s_McSerialNo:
            for k in s_ScType:
                if model.v_mothcoil[i,j,k].value is not None and model.v_mothcoil[i,j,k].value>0:
                        print(f" From Mother coil Type {i} having serial No {j} obtained slit {k} and : # of slits obtained is {model.v_mothcoil[i,j,k].value}")                

 From Mother coil Type 1550 having serial No sno - 1 obtained slit 350 and : # of slits obtained is 1.0
 From Mother coil Type 1550 having serial No sno - 1 obtained slit 600 and : # of slits obtained is 2.0
 From Mother coil Type 1550 having serial No sno - 2 obtained slit 550 and : # of slits obtained is 2.0
 From Mother coil Type 1550 having serial No sno - 2 obtained slit 140 and : # of slits obtained is 3.0
 From Mother coil Type 1550 having serial No sno - 4 obtained slit 550 and : # of slits obtained is 2.0
 From Mother coil Type 1550 having serial No sno - 4 obtained slit 220 and : # of slits obtained is 2.0
 From Mother coil Type 1550 having serial No sno - 5 obtained slit 350 and : # of slits obtained is 1.0
 From Mother coil Type 1550 having serial No sno - 5 obtained slit 600 and : # of slits obtained is 2.0
 From Mother coil Type 1550 having serial No sno - 7 obtained slit 550 and : # of slits obtained is 2.0
 From Mother coil Type 1550 having serial No sno - 7 obtained sl

In [13]:
model.v_mothcoilusd.pprint()

v_mothcoilusd : Size=40, Index=v_mothcoilusd_index
    Key                : Lower : Value : Upper : Fixed : Stale : Domain
     (1350, 'sno - 1') :     0 :   0.0 :     1 : False : False : Binary
    (1350, 'sno - 10') :     0 :   0.0 :     1 : False : False : Binary
    (1350, 'sno - 11') :     0 :   0.0 :     1 : False : False : Binary
    (1350, 'sno - 12') :     0 :   0.0 :     1 : False : False : Binary
    (1350, 'sno - 13') :     0 :   0.0 :     1 : False : False : Binary
    (1350, 'sno - 14') :     0 :   0.0 :     1 : False : False : Binary
    (1350, 'sno - 15') :     0 :   0.0 :     1 : False : False : Binary
    (1350, 'sno - 16') :     0 :   0.0 :     1 : False : False : Binary
    (1350, 'sno - 17') :     0 :   1.0 :     1 : False : False : Binary
    (1350, 'sno - 18') :     0 :   0.0 :     1 : False : False : Binary
    (1350, 'sno - 19') :     0 :   0.0 :     1 : False : False : Binary
     (1350, 'sno - 2') :     0 :   0.0 :     1 : False : False : Binary
    (1350, 's

In [14]:
model.capacity_constraint.pprint()

capacity_constraint : Size=40, Index=capacity_constraint_index, Active=True
    Key                : Lower : Body                                                                                                                                                                                                                  : Upper : Active
     (1350, 'sno - 1') :  -Inf :       550*v_mothcoil[1350,sno - 1,550] + 350*v_mothcoil[1350,sno - 1,350] + 220*v_mothcoil[1350,sno - 1,220] + 140*v_mothcoil[1350,sno - 1,140] + 600*v_mothcoil[1350,sno - 1,600] - 1350*v_mothcoilusd[1350,sno - 1] :   0.0 :   True
    (1350, 'sno - 10') :  -Inf : 550*v_mothcoil[1350,sno - 10,550] + 350*v_mothcoil[1350,sno - 10,350] + 220*v_mothcoil[1350,sno - 10,220] + 140*v_mothcoil[1350,sno - 10,140] + 600*v_mothcoil[1350,sno - 10,600] - 1350*v_mothcoilusd[1350,sno - 10] :   0.0 :   True
    (1350, 'sno - 11') :  -Inf : 550*v_mothcoil[1350,sno - 11,550] + 350*v_mothcoil[1350,sno - 11,350] + 220*v_mothcoil[1350,sno - 1

# Comments

In [15]:
# df_slit['Actual Demand'] = None
# for index,i in enumerate(df_slit['Demand (tons)']):
#     result = i/1550
#     result = round(result * 28)
#     df_slit.at[index,'Actual Demand'] = result

In [16]:
# def Utilization_rule(model,i,j,k):
#     return model.v_mothcoil[i,j,k] <= model.v_mothcoilusd[i,j] * 1000
# model.utilization_constraint = Constraint(s_McType,s_McSerialNo,s_ScType, rule = Utilization_rule)